# Curriculum 2: Designing Infrastructure and Managing Migration

## Lesson 2: Azure Cloud Capabilities

#### Terms
**SLA** (*Service Level Agreement*, *Соглашение об уровне обслуживания*) –– соглашение между компанией и облачным вендором по обеспечению % бесперебойной работы.  
**VPN** (*Virtual Private Network*, *Виртуальная частная сеть*) –– технология, позволяющая безопасно передавать данные из одной сети в другую. Это удобно при удаленном доступе.  
**High Availability** (*Высокая доступность*) –– внедрение различных технологий с целью минимизировать время простоя.  
**VM** (*Virtual Machines*, *Виртуальная машина*) –– эмуляция системы, которая использует вычислительные мощности, а не физические.  

### Concept 6: Exercise, Creating Web Servers

(**VM**) После создания учебного окружения на 2 часа, можно использовать login и pass для доступа в портал Azure.

1. Azure Portal: https://portal.azure.com/
2. Выбираем Virtual Machine
3. Заполняем обязательные пункты (VM name, Azure Resource Group (RSG), Subscription, Region, VM size, credentials...)
4. Заполняем необязательные пункты (Auto-shutdown, Data Disk, Load Balancing, tags...)
5. Review + Create

Порты:
- HTTP 80
- HTTPS 443
- SSL 22
- RDP 3389

После создания VM можно подключиться с помощью RDP к удаленному рабочему столу:
- VM Web01 -> Connect -> RDP 
- Download RDP file
- Или просто указываем в программе public IP + port (login+pass указывали ранее)

(**IIS**) После подключения идем в Server Manager -> Add role and Features Wizard:
- Шаг Server roles -> Включаем Web server (IIS)
- Next, next, next, install

В панеле управления можно найти IIS и расположение файлов сайта, которые открываются по публичному IP VM.

### Concept 8: High Availability

Azure HA details:
- Availability zones –– регионы и различные дата-центры
- VM scale sets –– кастомизация настроек VM (ядра, доменты и тд)
- Availability sets
    - Fault domains
    - Update domains

Load balancing:
- Application gateway
- Front door
- Azure load balance
- Traffic manager

**Scale sets** –– service that combine a group of VMs to be rus as a single unit.  
**Availability set** –– strategic grouping of VMs resources для обеспечения отсутствия single point поражения в ЦОД.



### Concept 9: Use Load Balancer to Distribute Web Traffic

Так как у учебного окружения тайм-аут в 2ч, созданные ранее VM недоступны и необходимо пересоздавать всё заново.

#### Create the Load balancer

1. Azure Portal: https://portal.azure.com/
2. Выбираем Load balancer
3. Basics: Resource group, Instance name
4. Frontend IP configuration: add Frontend IP configuration -> create new Public IP address
5. Review and Create -> Create
6. Ищем на портале Virtual networks -> create Virtual network
7. Basics: Resource group (from step 2), Instance name
8. IP addresses: не должно быть конфликтов
9. Review and Create -> Create

#### Create the First VM for the Backend Pool

1. Azure Portal: https://portal.azure.com/
2. Virtual Machines -> create VM
3. Basics: 
    - Project details: 
        - Resource group
    - Instance details:  
        - Instance name
        - Availability options = `Availability zone` + Availability zone = `1`
        - Image = `Windows server 2019 GEN 1` (можно изм gen 2 на gen 1 нажав configure VM generation)
        - Size = `*DS1_v2*`
    - Administratoc account: 
        - loging + pass
    - Inbound ports: 
        - HTTP, HTTPS, RDP
4. Networking: 
    - Network interface: 
        - Virtual network & Subnet & Public IP (filled from Load balancer)
        - NIC network security group = `Advanced`
        - Configure network security group (filled from Load balancer)
    - Load balancing: 
        - Check In Place this virtual machine behind an existing load balancing solution? for true
    - Load balancing settings: 
        - Load balancing options = `Azure load balancer`
        - Load balancer = `weblb` (from Load balancer)
        - ??create Backend pool = `webbackendpool`??
5. Management:
    - Monitoring: 
        - Boot diagnostic `disable`
6. Review and Create -> Create

#### Create the Second VM for the Backend Pool

1. Azure Portal: https://portal.azure.com/
2. Virtual Machines -> create VM
3. Basics: simmilar to the First VM, only Availability zone = `2` (not 1)
4. Networking: simmilar to the First VM
5. Management: Boot diagnostic `disable`
6. Review and Create -> Create

#### Modify your Network Security Groups for the Backend Pools

1. Azure Portal: https://portal.azure.com/
2. Network Security Groups (уже созданы для каждой VM, в classic - нет) -> web01lb-nsg (уже создана) -> Settings: 
3. add Inbound security rules (for the both VMs):
    - Source = `service tag` 
    - Source service tag = `Internet`
    - Source port ranges `*`
    - Destination `any`
    - Destination port ranges `80`
    - Protocol `TCP`
    - Action `Allow`
    - Priority `100`
    - Name `HTTP`
4. Add IIS for both VMs: 
    - connect by RDP (`VMsIP:3389) to VM1, VM2 and config IIS like in CU02LE02CO03
    - create `index.html` for both

#### Establish the Network the Backend Pool VMs

1. Azure Portal: https://portal.azure.com/
2. Virtual machines -> web01lb -> Settings -> Networking
3. Load balancing: add load balancing (на самом деле я создал их еще в меню создания VM)
    - Load balancing options `Azure load balancer`
    - Load balancer `weblb` (создавали ранее)
    - new Backend pool
4. Аналогично для web02lb


#### Setup the Healthprobe

Healthprobe проверит каждый веб-сервер и приостановит маршрутизацию трафика на сервер, который не отвечает.

1. Azure Portal: https://portal.azure.com/
2. Load balancers -> `weblb` (уже созданный) -> Settings -> Health probes 
3. Add Health probe: 
    - Name `Healthprobe1`
    - Protocol `HTTP`
    - Port `80`
    - Path `/`
    - Interval `5`
    - Unhealthy threshold `2`
4. Load balancers -> `weblb` (уже созданный) -> Settings -> Load balancer rules
5. Add load balancer rule: 
    - Name `httptowebserver`
    - IP `IPv4`
    - Frontend IP check `loadbalancerIP`
    - Backend pool check `webbackendpool`
    - Protocol `TCP`
    - Port `80`
    - Backend port `80`
    - Healthprobe check `healthprobe1`
    - Session persistence `None`
    - Timeout `4`
    - TCP reset `Disable`
    - Floating IP `Disable`
    - ...SNAT check (recomend).

#### Confirm Everything is Working

1. Azure Portal: https://portal.azure.com/
2. Load balancers -> `weblb` -> Overview -> в дашборде see more -> Public IP address
3. Отключаем IIS и трафик перенаправляется на другую VM.

### Concept 12: Quiz, Application services

**Load Balancing**: traffic manager, application gateway, исп как серверы угроз.  
**VM scale set**: позволяет приложению автоматически масштабироваться, все VM в одном SS настраиваются и управляются как одно целое.  

### Concept 13: Exercise, Create a DNN Web App 

На Виртуальных машинах Azure имя распределенной сети (DNN) маршрутизирует трафик в соответствующий кластерный ресурс. Оно обеспечивает более простой способ подключения к экземпляру отказоустойчивого кластера (FCI) SQL Server, чем имя виртуальной сети (VNN), и не требует использования Azure Load Balancer.

1. Azure Portal: https://portal.azure.com/
2. DNN Platform:
    - Name: `DNN<unique>`
    - Resource group: choose exists
    - SQL Database -> create Database (я случайно отменил создание БД и поэтому пришлось отдельно создавать в разделе БД, но в рамках DNN создание БД выглядит менее избыточно):
        - Name: DNNDB
        - Target server: new server
            - Name server: `DNN<unique>`
            - Login + Pass
        - Pricing tier: Standard S1, 20 DTU, 250 gb 
3. Deploy=Create (создается довольно много позиций)
4. Go to the created DNN app service -> Overview -> copy URL -> open URL (потребуется некоторое время)
5. fill data:
    - Administrative Information:
        - Username: `host` (для прода получше что-нибудь)
    - Database Information:
        - Database Type: chose `SQL Server/SQL server express database`
        - Server name: `DNN<unique>.database.windows.net`
        - DB Name: `DNNDB` (из пред шага)
        - Security: choose `User defined`
        - DB login + DB pass
        - check in `Database owner`
6. Visit Website `https://dnn<unique>.azurewebsites.net`

В дополнение: можно перейти в SQL DB и посмотреть что хранится в БД. Адрес БД примерно `dnn<unique>.database.windows.net`. При попытке подключиться с локального компа, потребуется внести IP в whitelist (просто нажимаем на гипертекст ошибки при попытке перехода в Query editor).

### Concept 15: Architecting Solution to Meet SLA Requirements

Design patterns: design principles:
- Design of self-healing
- Design for redundancy
- Design to scale out
- Design for evolution
- Design for operations
- Managed services
- Focus on business needs

Types of styles:
- Big computer
- Big data
- Event-driven architecture
- Microservices

### Concept 16: Communication Design Between On-premise and Cloud

You will need to consider a few different factors when designing a network solution:
- Latency
- Security
- Speed
- Redundancy

And you will be using tools to communicate between:
- VNET/VNET Peering
- Gateway
- VPN
- Network Watcher

These tools determine the method:
- Using VNTE
- Virtual Network Endpoints
- VNET Peering
- Private Link

Here are some ways to communicate with On-premise:
- Point-to-Site Virtual Private Network (VPN)
- Site-to-Site VPN
- Azure ExpressRoute

### Concept 17: Route Based VPNs

<img src="imgs/cu02le02co17.png" title="Инструменты и среда" width="400" height="400" />


### COncept 18: Quiz, Route Based VPNs

**VN peering** –– Methods to communicate among Azure resources.  
**ExpressRoute** –– Methods to securely communicate with on-premises networks.  
**Network Security group** –– Used to filter traffic to and from other Azure resources.

### Concept 19: Exercise, Create VPN Gateway

Создадим VPN шлюз, который необходим для соединения локальной среды с Azure. Полезно, когда нет необходимости ресурсам Azure быть доступными в общем интернете.

1. Azure Portal: https://portal.azure.com/
2. Virtual networks -> create:
    - Basics:
        - Resource group: check in
        - Name: `vpngw1`
    - IP addresses: all default
    - Security: all `disable`
3. Virtual network gateways -> Create:
    - Basics (instance):
        - Name: `mynetworkgw1`
        - Gateway type: `VPN`
        - VPN type: `Route based`
        - SKU: `VpnGW1` + Generation: `Generation1`
        - Virtual network: recently early created
        - Gateway subnet address range: as default `10.0.1.0/24` (2 point, 256 addresses)
    - Basics (public IP address):
        - `create new`
        - name: `vpnGW1IP`
        - Enable active-active mode, Configure BGP: `disable`
4. Go to resource -> Public IP address

### Concept 21: Network Security Group Modifications

Группа безопасности сети Azure (Network Security Group) позволяет фильтровать входящий и исходящий трафик ресурсов Azure в виртуальной сети Azure. В этой группе содержатся правила безопасности, которые разрешают или запрещают исходящий и входящий трафик нескольких типов ресурсов Azure. 

NSG похож на брандмауэр, но имеет только базовые функции. Правила NSG будут иметь приоритет над правилами брандмауэра. Дублирование правил тоже не лучшая стратегия.

### Concept 23: Exercise, Use NSG to open ports for FTP

1. Azure Portal: https://portal.azure.com/
2. Create VM (concept 6)
3. Go to VM -> copy Public IP address -> connect via RDP port 3389
4. Create IIS (concept 6)
    - Server roles: When check IIS при развороте `Web Server (IIS)` проставляются не все галки. Внутри будет пункт `FTP server`
    - Features: В следующем окне `Telnet client`
5. IIS -> `web01` -> right mouse Add FTP site
    - Name: `myftpsite`
    - Path: `c:\inetpub\wwwroot`
    - Binding and SSL:
        - IP Address: `all unsigned`
        - Port: `21`
        - check Start FTP automatically
        - SSL: `required`
        - SSL certificate: tenant encryptioncert
    - Authentication:
        - Authentication: check in `Basic`
        - Allow access to: choose `All users`
        - Permissions: Select both `Read` and `Write`
6. Теперь можно подключиться к FTP внутри VM, но не извне. CMD -> `telnet localhost 21`. Теперь нам нужно перейти к настройке сервера для внешнего доступа, изменив внутренний брандмауэр, а также правила Azure NSG.
7. IIS -> FTP Firewall Support
    - Вносим Public IP из VM Azure portal
    - `net stop ftpsvc` stop FTP 
    - `net start ftpsvc` run FTP
8. Portal -> VM -> Networking -> add Inbound port rule:
    - Port range: `21, 5000-6000`
    - Name: `FTPservices`
9. Test FTP by Filezila, к примеру   